In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [6]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000155                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000263
0.000750      2.0 -0.000126                  2.0 -0.000354
0.001052      3.0 -0.000053                  3.0 -0.000154
0.001476      4.0  0.000048                  4.0 -0.000083
0.002070      5.0  0.000179                  5.0  0.000002
0.002904      6.0  0.000348                  6.0  0.000107
0.004074      7.0  0.000562                  7.0  0.000244
0.005714      8.0  0.000830                  8.0  0.000426
0.008015      9.0  0.001162                  9.0  0.000654
0.011243     10.0  0.001570                 10.0  0.000932
0.015771     11.0  0.002067                 11.0  0.001269
0.022122     12.0  0.002670                 12.0  0.001675
0.031031     13.0  0.003391                 13.0  0.002162
0.043528     14.0  0.004146                 14.0  0.002669
0.061057     15.0  0.004815                 15.0  0.003120
0.085645     16.0  0.005448                 16.0  0.003555
0.120136     17.0  0.006138                 17.0  0.004026
0.168516     18.0  0.006890                 18.0  0.004524
0.236378     19.0  0.007711                 19.0  0.005045
0.331549     20.0  0.008600                 20.0  0.005592
0.465100     21.0  0.009457                 21.0  0.006169
0.652400     22.0  0.008676                 22.0  0.005794
0.915100     23.0  0.006141                 23.0  0.004175
1.283650     24.0  0.003864                 24.0  0.002649
1.800600     25.0  0.002291                 25.0  0.001571
2.525700     26.0  0.001237                 26.0  0.000838
3.542800     27.0  0.000554                 27.0  0.000353
4.969550     28.0  0.000173                 28.0  0.000077
6.970850     29.0 -0.000012                 29.0 -0.000062
9.778100     30.0 -0.000120                 30.0 -0.000146
13.715850    31.0 -0.000195                 31.0 -0.000207
19.239350    32.0 -0.000246                 32.0 -0.000248
26.987250    33.0 -0.000265                 33.0 -0.000264
37.855300    34.0 -0.000261                 34.0 -0.000258
53.100050    35.0 -0.000253                 35.0 -0.000262
73.887500    36.0 -0.000247                 36.0 -0.000267
97.662500    37.0 -0.000247                 37.0 -0.000264
121.437500   38.0 -0.000259                 38.0 -0.000265
145.212500   39.0 -0.000279                 39.0 -0.000268
168.987500   40.0 -0.000298                 40.0 -0.000271
192.762500   41.0 -0.000314                 41.0 -0.000275
216.537500   42.0 -0.000329                 42.0 -0.000286
240.312500   43.0 -0.000343                 43.0 -0.000299
264.087500   44.0 -0.000356                 44.0 -0.000310
287.862500   45.0 -0.000365                 45.0 -0.000316
311.637500   46.0 -0.000342                 46.0 -0.000294
335.412500   47.0 -0.000260                 47.0 -0.000213
359.187500   48.0 -0.000120                 48.0 -0.000085
382.962500   49.0  0.000076                 49.0  0.000086
406.737500   50.0  0.000355                 50.0  0.000322
430.512500   51.0  0.000742                 51.0  0.000640
454.287500   52.0  0.001271                 52.0  0.001073
478.062500   53.0  0.001971                 53.0  0.001654
501.837500   54.0  0.002870                 54.0  0.002416
525.612500   55.0  0.003985                 55.0  0.003335
549.387500   56.0  0.005319                 56.0  0.004462
573.162500   57.0  0.006859                 57.0  0.005802
596.937500   58.0  0.008571                 58.0  0.007338
620.712500   59.0  0.010408                 59.0  0.009041
644.487500   60.0  0.012312                 60.0  0.010808
668.262500   61.0  0.014225                 61.0  0.012586
692.037500   62.0  0.016095                 62.0  0.014369
715.812500   63.0  0.017880                 63.0  0.016111
739.587500   64.0  0.019550                 64.0  0.017777
76

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -32.747664  0.000000e+00 -32.747664        NaN   
1.000000e-08 1            NaN           NaN        NaN -32.762517   
6.244000e-04 2     -32.747664  3.533345e-08 -32.747664 -32.762517   
8.759000e-04 3     -32.747664  5.092928e-08 -32.747664 -32.762517   
1.228600e-03 4     -32.747664  7.719870e-08 -32.747664 -32.762517   
1.723400e-03 5     -32.747664  1.214675e-07 -32.747664 -32.762517   
2.417400e-03 6     -32.747664  1.958874e-07 -32.747664 -32.762517   
3.390900e-03 7     -32.747664  3.207319e-07 -32.747664 -32.762518   
4.756500e-03 8     -32.747664  5.296845e-07 -32.747664 -32.762518   
6.672000e-03 9     -32.747665  8.785336e-07 -32.747664 -32.762518   
9.358900e-03 10    -32.747665  1.459702e-06 -32.747663 -32.762518   
1.312780e-02 11    -32.747665  2.425853e-06 -32.747663 -32.762518   
1.841450e-02 12    -32.747665  4.028688e-06 -32.747661 -32.762519   
2.583020e-02 13    -32.747666  6.682259e-06 -32.747659 -32.762519   
3.623230e-02 14    -32.747666  1.106324e-05 -32.747655 -32.762520   
5.082340e-02 15    -32.747666  1.814879e-05 -32.747648 -32.762520   
7.129060e-02 16    -32.747665  2.922940e-05 -32.747636 -32.762521   
1.000000e-01 17    -32.747664  4.625490e-05 -32.747617 -32.762521   
1.402710e-01 18    -32.747660  7.235875e-05 -32.747588 -32.762521   
1.967600e-01 19    -32.747654  1.122915e-04 -32.747542 -32.762519   
2.759970e-01 20    -32.747643  1.732556e-04 -32.747470 -32.762515   
3.871000e-01 21    -32.747622  2.660238e-04 -32.747356 -32.762508   
5.431000e-01 22    -32.747587  4.056581e-04 -32.747182 -32.762493   
7.617000e-01 23    -32.747544  5.866388e-04 -32.746957 -32.762477   
1.068500e+00 24    -32.747512  7.782820e-04 -32.746734 -32.762472   
1.498800e+00 25    -32.747503  9.665340e-04 -32.746537 -32.762488   
2.102400e+00 26    -32.747522  1.149343e-03 -32.746373 -32.762528   
2.949000e+00 27    -32.747574  1.325190e-03 -32.746249 -32.762597   
4.136600e+00 28    -32.747665  1.493495e-03 -32.746171 -32.762699   
5.802500e+00 29    -32.747800  1.662814e-03 -32.746137 -32.762843   
8.139200e+00 30    -32.747992  1.851319e-03 -32.746140 -32.763040   
1.141700e+01 31    -32.748261  2.074003e-03 -32.746187 -32.763308   
1.601470e+01 32    -32.748636  2.342899e-03 -32.746293 -32.763673   
2.246400e+01 33    -32.749158  2.677980e-03 -32.746481 -32.764172   
3.151050e+01 34    -32.749907  3.142266e-03 -32.746764 -32.764867   
4.420010e+01 35    -32.751014  3.857382e-03 -32.747156 -32.765855   
6.200000e+01 36    -32.752665  4.975411e-03 -32.747690 -32.767365   
8.577500e+01 37    -32.755010  6.624023e-03 -32.748386 -32.769582   
1.095500e+02 38    -32.757474  8.393070e-03 -32.749081 -32.771924   
1.333250e+02 39    -32.760018  1.020646e-02 -32.749812 -32.774316   
1.571000e+02 40    -32.762634  1.203448e-02 -32.750599 -32.776704   
1.808750e+02 41    -32.765315  1.387744e-02 -32.751437 -32.779092   
2.046500e+02 42    -32.768061  1.573848e-02 -32.752322 -32.781499   
2.284250e+02 43    -32.770863  1.761406e-02 -32.753249 -32.783981   
2.522000e+02 44    -32.773719  1.950453e-02 -32.754214 -32.786557   
2.759750e+02 45    -32.776630  2.141371e-02 -32.755217 -32.789212   
2.997500e+02 46    -32.779609  2.336543e-02 -32.756244 -32.791943   
3.235250e+02 47    -32.782722  2.551373e-02 -32.757208 -32.794802   
3.473000e+02 48    -32.786053  2.811200e-02 -32.757941 -32.797814   
3.710750e+02 49    -32.789617  3.133740e-02 -32.758279 -32.801012   
3.948500e+02 50    -32.793496  3.543099e-02 -32.758065 -32.804489   
4.186250e+02 51    -32.797775  4.070878e-02 -32.757066 -32.808298   
4.424000e+02 52    -32.802495  4.752023e-02 -32.754975 -32.812445   
4.661750e+02 53    -32.807689  5.629302e-02 -32.751396 -32.816985   
4.899500e+02 54    -32.813365  6.751863e-02 -32.745846 -32.821945   
5.137250e+02 55    -32.8

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')